In [1]:
import os
import re
import glob
from langchain.schema import Document
from langchain.document_loaders import PyPDFLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter, SentenceTransformersTokenTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_core.language_models.fake import FakeListLLM  # Importa el LLM falso

In [2]:
# --- Configuración ---
DOCUMENT_PATH = "./data"
FAISS_INDEX_PATH = "./my_faiss_index"
EMBEDDINGS_MODEL = "all-MiniLM-L6-v2"  # Puedes probar "sentence-transformers/all-mpnet-base-v2" o uno específico
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 100
USE_SEMANTIC_CHUNKING = True  # Cambiar a False para usar RecursiveCharacterTextSplitter

In [8]:
def clean_text(text: str) -> str:
    # 1) Eliminar cabeceras/pies de página comunes
    text = re.sub(r"Página\s*\d+\s*/\s*\d+", " ", text, flags=re.IGNORECASE)
    text = re.sub(r"Manual de Usuario|Funcionalidad MiCoto", " ", text)

    # 2) Deshacer guiones al final de renglón: “co-\nmentario” → “comentario”
    text = re.sub(r"(\w)-\s*\n\s*(\w)", r"\1\2", text)

    # 3) Unir saltos de línea dentro del mismo párrafo
    text = re.sub(r"(?<!\n)\n(?!\n)", " ", text)

    # 4) Quitar URLs, correos y teléfonos
    text = re.sub(r"\bhttps?://\S+\b", " ", text)
    text = re.sub(r"\b[\w\.-]+@[\w\.-]+\.\w{2,}\b", " ", text)
    text = re.sub(r"\b\d{2,4}[-\s]?\d{2,4}[-\s]?\d{2,4}\b", " ", text)

    # 5) Unificar comillas y guiones largos
    text = text.replace("“", '"').replace("”", '"').replace("—", "-")
    # 6) Colapsar múltiples espacios y líneas en blanco
    text = re.sub(r"[ \t]{2,}", " ", text)
    text = re.sub(r"\n{3,}", "\n\n", text)

    return text.strip()

In [3]:
def is_useful_for_rag(text: str) -> bool:
    t = text.strip()
    return len(t) > 50 and len(t.split()) >= 5

In [4]:
def load_pdfs_and_txts(path: str):
    docs = []
    # PDFs
    for pdf_path in glob.glob(os.path.join(path, "*.pdf")):
        loader = PyPDFLoader(pdf_path)
        for d in loader.load():
            txt = clean_text(d.page_content)
            if is_useful_for_rag(txt):
                d.page_content = txt
                d.metadata["source"] = os.path.basename(pdf_path)
                docs.append(d)

    # TXT (opcional)
    for txt_path in glob.glob(os.path.join(path, "*.txt")):
        loader = TextLoader(txt_path, encoding="utf-8")
        for d in loader.load():
            txt = clean_text(d.page_content)
            if is_useful_for_rag(txt):
                d.page_content = txt
                d.metadata["source"] = os.path.basename(txt_path)
                docs.append(d)

    # Deduplicación
    seen, unique = set(), []
    for d in docs:
        h = hash(d.page_content)
        if h not in seen:
            seen.add(h)
            unique.append(d)

    print(f"→ Documentos únicos cargados: {len(unique)}")
    return unique

In [5]:
def create_rag_index():
    # 1) Carga y limpieza
    documents = load_pdfs_and_txts(DOCUMENT_PATH)

    # 2) Chunking
    if USE_SEMANTIC_CHUNKING:
        splitter = SentenceTransformersTokenTextSplitter(chunk_overlap=CHUNK_OVERLAP, chunk_size=CHUNK_SIZE)
        chunks = splitter.split_documents(documents)
        print(f"→ Se generaron {len(chunks)} chunks (usando Sentence Transformers for chunking).")
    else:
        splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)
        chunks = splitter.split_documents(documents)
        print(f"→ Se generaron {len(chunks)} chunks (usando RecursiveCharacterTextSplitter).")

    # 3) Embeddings
    embeddings = HuggingFaceEmbeddings(
        model_name=EMBEDDINGS_MODEL,
        model_kwargs={"device": "cpu"}
    )
    print(f"→ Modelo de embeddings cargado: {EMBEDDINGS_MODEL}")

    # 4) FAISS
    vector_store = FAISS.from_documents(chunks, embeddings)
    vector_store.save_local(FAISS_INDEX_PATH)
    print(f"✅ Índice FAISS creado en: {FAISS_INDEX_PATH}")
    return vector_store

In [6]:
def test_rag(vector_store):
    """Función simple para probar el RAG localmente."""
    # Usamos un LLM falso para probar la recuperación
    llm = FakeListLLM(responses=["Respuesta basada en el contexto."])

    prompt_template = """Eres un asistente útil. Responde la pregunta basándote únicamente en el siguiente contexto.
    Si la respuesta no está en el contexto, di "No tengo suficiente información para responder a eso". No inventes.

    Contexto: {context}

    Pregunta: {question}

    Respuesta:"""
    QA_CHAIN_PROMPT = PromptTemplate.from_template(prompt_template)

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,  # Pasa la instancia del LLM falso
        retriever=vector_store.as_retriever(),
        chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
        return_source_documents=True
    )

    while True:
        query = input("Pregunta al RAG (o escribe 'salir'): ")
        if query.lower() == 'salir':
            break

        result = qa_chain({"query": query})
        print("\nRespuesta:", result["result"])
        print("\nFuentes:")
        for doc in result["source_documents"]:
            print(f"- {doc.metadata['source']}: {doc.page_content[:100]}...") # Mostrar los primeros 100 caracteres
        print("\n" + "="*50 + "\n")

In [9]:
os.makedirs(DOCUMENT_PATH, exist_ok=True)
vector_store = create_rag_index()
if vector_store:
    print("\n--- ¡Índice creado! Ahora puedes probar el RAG localmente: ---")

Ignoring wrong pointing object 300 0 (offset 0)


→ Documentos únicos cargados: 21


2025-05-11 00:06:26.898823: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/opt/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


→ Se generaron 30 chunks (usando Sentence Transformers for chunking).


/var/folders/wj/697d4ndd5cnd828k3szpyr0c0000gn/T/ipykernel_32661/542114483.py:16: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


→ Modelo de embeddings cargado: all-MiniLM-L6-v2
✅ Índice FAISS creado en: ./my_faiss_index

--- ¡Índice creado! Ahora puedes probar el RAG localmente: ---


In [10]:
test_rag(vector_store)

/var/folders/wj/697d4ndd5cnd828k3szpyr0c0000gn/T/ipykernel_32661/3442367660.py:28: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": query})



Respuesta: Respuesta basada en el contexto.

Fuentes:
- Manual de Usuario de la Plataforma _Mi Coto_.pdf: requisitos para recibir notificaciones : 1. para recibir notificaciones, debe estar dado de alta en ...
- Manual de Usuario de la Plataforma _Mi Coto_.pdf: manual de usuario de la plataforma " mi coto " 1. introduccion ● 1. 1 descripcion general de la plat...
- Manual de Usuario de la Plataforma _Mi Coto_.pdf: [UNK] 1. 3. 5 mensaje de bienvenida : un mensaje de bienvenida para el usuario. 2. mi cuenta ● 2. 1 ...
- ManualdeUsuario.pdf: paso 2 – proporciona tu nombre de usuario y pasword en nuestro sitio web dentro de nuestra aplicacio...


